In [1]:
spark.conf.set("spark.sql.repl.eagerEval.enabled",True)

In [2]:
import os
import subprocess
import pandas as pd
from pyspark.sql import functions as F
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [3]:
directory = 'gs://msca-bdp-tweets/final_project/'
# file = '*.json'
# path = directory + file
path = directory

In [4]:
cmd = 'hadoop fs -du -s -h ' + directory

p = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, universal_newlines=True)
for line in p.stdout.readlines():
    print (line)
    
retval = p.wait()

497.0 G  497.0 G  gs://msca-bdp-tweets/final_project



In [5]:
%%time

tweets_df = spark.read.json(path)

22/12/07 16:06:41 WARN org.apache.spark.sql.execution.datasources.SharedInMemoryCache: Evicting cached table partition metadata from memory due to size constraints (spark.sql.hive.filesourcePartitionFileCacheSize = 262144000 bytes). This may impact query planning performance.


CPU times: user 1.45 s, sys: 301 ms, total: 1.75 s
Wall time: 7min 53s


22/12/07 16:13:25 WARN org.apache.spark.sql.catalyst.util.package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [6]:
%%time

tweets_df.count()

CPU times: user 526 ms, sys: 144 ms, total: 670 ms
Wall time: 2min 42s


99992797

In [7]:
tweets_df.printSchema()

root
 |-- coordinates: struct (nullable = true)
 |    |-- coordinates: array (nullable = true)
 |    |    |-- element: double (containsNull = true)
 |    |-- type: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- display_text_range: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- entities: struct (nullable = true)
 |    |-- hashtags: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- indices: array (nullable = true)
 |    |    |    |    |-- element: long (containsNull = true)
 |    |    |    |-- text: string (nullable = true)
 |    |-- media: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- additional_media_info: struct (nullable = true)
 |    |    |    |    |-- description: string (nullable = true)
 |    |    |    |    |-- embeddable: boolean (nullable = true)
 |    |    |    |    |-- monetizable: boolean (nullable = true)
 |    |    |    |   

### Read Nested Data

In [6]:
tweets_df_filtered = tweets_df[['user','coordinates','created_at','id','text','favorite_count','retweet_count','retweeted_status','reply_count','quote_count','lang','place']]

In [7]:
tweets_df_filtered

user,coordinates,created_at,id,text,favorite_count,retweet_count,retweeted_status,reply_count,quote_count,lang,place
"{false, Thu Mar 0...",null,Tue May 24 22:09:...,1529223198327287808,RT @ABC: “Why are...,0,0,"{null, Tue May 24...",0,0,en,null
"{false, Thu May 1...",null,Tue May 24 22:09:...,1529223197253435392,Indiana High Scho...,0,0,null,0,0,en,null
"{false, Fri Jun 2...",null,Tue May 24 22:09:...,1529223198478311425,RT @jaketapper: F...,0,0,"{null, Tue May 24...",0,0,en,null
"{false, Wed Feb 2...",null,Tue May 24 22:09:...,1529223198449012736,#Uvalde is just a...,0,0,null,0,0,en,null
"{false, Wed Jul 0...",null,Tue May 24 22:09:...,1529223198683942914,RT @Josh_Moon: 14...,0,0,"{null, Tue May 24...",0,0,en,null
"{false, Thu Feb 1...",null,Tue May 24 22:09:...,1529223198675546112,RT @Jim_Jordan: L...,0,0,"{null, Tue May 24...",0,0,en,null
"{false, Mon Mar 1...",null,Tue May 24 22:09:...,1529223199036059648,RT @meganbang3: M...,0,0,"{null, Mon May 23...",0,0,en,null
"{false, Fri May 3...",null,Tue May 24 22:09:...,1529223199032066050,RT @jewishaction:...,0,0,"{null, Tue May 24...",0,0,en,null
"{false, Fri Jul 2...",null,Tue May 24 22:09:...,1529223199422316545,RT @LRiddickESPN:...,0,0,"{null, Tue May 24...",0,0,en,null
"{false, Thu Sep 0...",null,Tue May 24 22:09:...,1529223199174672385,@TheUSASingers My...,0,0,null,0,0,en,null


In [8]:
from pyspark.sql.functions import col


def flatten_df(nested_df):
    stack = [((), nested_df)]
    columns = []

    while len(stack) > 0:
        parents, df = stack.pop()

        flat_cols = [
            col(".".join(parents + (c[0],))).alias("_".join(parents + (c[0],)))
            for c in df.dtypes
            if c[1][:6] != "struct"
        ]

        nested_cols = [
            c[0]
            for c in df.dtypes
            if c[1][:6] == "struct"
        ]

        columns.extend(flat_cols)

        for nested_col in nested_cols:
            projected_df = df.select(nested_col + ".*")
            stack.append((parents + (nested_col,), projected_df))

    return nested_df.select(columns)

In [9]:
tweets_df_flattened = flatten_df(tweets_df_filtered)

In [10]:
tweets_df_flattened

created_at,id,text,favorite_count,retweet_count,reply_count,quote_count,lang,place_country,place_country_code,place_full_name,place_id,place_name,place_place_type,place_url,place_bounding_box_coordinates,place_bounding_box_type,retweeted_status_created_at,retweeted_status_display_text_range,retweeted_status_favorite_count,retweeted_status_favorited,retweeted_status_filter_level,retweeted_status_id,retweeted_status_id_str,retweeted_status_in_reply_to_screen_name,retweeted_status_in_reply_to_status_id,retweeted_status_in_reply_to_status_id_str,retweeted_status_in_reply_to_user_id,retweeted_status_in_reply_to_user_id_str,retweeted_status_is_quote_status,retweeted_status_lang,retweeted_status_possibly_sensitive,retweeted_status_quote_count,retweeted_status_quoted_status_id,retweeted_status_quoted_status_id_str,retweeted_status_reply_count,retweeted_status_retweet_count,retweeted_status_retweeted,retweeted_status_source,retweeted_status_text,retweeted_status_truncated,retweeted_status_withheld_copyright,retweeted_status_withheld_in_countries,retweeted_status_user_contributors_enabled,retweeted_status_user_created_at,retweeted_status_user_default_profile,retweeted_status_user_default_profile_image,retweeted_status_user_description,retweeted_status_user_favourites_count,retweeted_status_user_followers_count,retweeted_status_user_friends_count,retweeted_status_user_geo_enabled,retweeted_status_user_id,retweeted_status_user_id_str,retweeted_status_user_is_translator,retweeted_status_user_listed_count,retweeted_status_user_location,retweeted_status_user_name,retweeted_status_user_profile_background_color,retweeted_status_user_profile_background_image_url,retweeted_status_user_profile_background_image_url_https,retweeted_status_user_profile_background_tile,retweeted_status_user_profile_banner_url,retweeted_status_user_profile_image_url,retweeted_status_user_profile_image_url_https,retweeted_status_user_profile_link_color,retweeted_status_user_profile_sidebar_border_color,retweeted_status_user_profile_sidebar_fill_color,retweeted_status_user_profile_text_color,retweeted_status_user_profile_use_background_image,retweeted_status_user_protected,retweeted_status_user_screen_name,retweeted_status_user_statuses_count,retweeted_status_user_translator_type,retweeted_status_user_url,retweeted_status_user_verified,retweeted_status_user_withheld_in_countries,retweeted_status_scopes_followers,retweeted_status_scopes_place_ids,retweeted_status_quoted_status_permalink_display,retweeted_status_quoted_status_permalink_expanded,retweeted_status_quoted_status_permalink_url,retweeted_status_quoted_status_created_at,retweeted_status_quoted_status_display_text_range,retweeted_status_quoted_status_favorite_count,retweeted_status_quoted_status_favorited,retweeted_status_quoted_status_filter_level,retweeted_status_quoted_status_id,retweeted_status_quoted_status_id_str,retweeted_status_quoted_status_in_reply_to_screen_name,retweeted_status_quoted_status_in_reply_to_status_id,retweeted_status_quoted_status_in_reply_to_status_id_str,retweeted_status_quoted_status_in_reply_to_user_id,retweeted_status_quoted_status_in_reply_to_user_id_str,retweeted_status_quoted_status_is_quote_status,retweeted_status_quoted_status_lang,retweeted_status_quoted_status_possibly_sensitive,retweeted_status_quoted_status_quote_count,retweeted_status_quoted_status_quoted_status_id,retweeted_status_quoted_status_quoted_status_id_str,retweeted_status_quoted_status_reply_count,retweeted_status_quoted_status_retweet_count,retweeted_status_quoted_status_retweeted,retweeted_status_quoted_status_source,retweeted_status_quoted_status_text,retweeted_status_quoted_status_truncated,retweeted_status_quoted_status_withheld_in_countries,retweeted_status_quoted_status_user_contributors_enabled,retweeted_status_quoted_status_user_created_at,retweeted_status_quoted_status_user_default_profile,retweeted_status_quoted_status_user_default_profile_image,retweeted_status_quoted_status_user_description,retweeted

### I. Tweets Clean-up and Filtering

In [11]:
tweets_df_en = tweets_df_flattened.filter(tweets_df_flattened.lang == 'en')
tweets_df_en = tweets_df_en.filter(lower(col('text')).contains('higher education')|\
                                   lower(col('text')).contains('k-12')|\
                                   lower(col('text')).contains('teachers')|\
                                   lower(col('text')).contains('primary school')|\
                                   lower(col('text')).contains('high school')|\
                                   lower(col('text')).contains('secondary school')|\
                                   lower(col('text')).contains('curriculum')|\
                                   lower(col('text')).contains('college')|\
                                   lower(col('text')).contains('university')|\
                                   lower(col('text')).contains('educational')|\
                                   lower(col('text')).contains('tuition')|\
                                   lower(col('text')).contains('student')|\
                                   lower(col('text')).contains('homework')|\
                                   lower(col('text')).contains('classroom')|\
                                   lower(col('text')).contains('research'))
#Filter out school shooting tweets
#tweets_df_en = tweets_df_en.filter(~lower(col('text')).contains('kill')&\
#                                   ~lower(col('text')).contains('shooting')&\
#                                   ~lower(col('text')).contains('shootings')&\
#                                   ~lower(col('text')).contains('shoot')&\
#                                   ~lower(col('text')).contains('gun')&\
#                                   ~lower(col('text')).contains('dead')&\
#                                   ~lower(col('text')).contains('murder')&\
#                                   ~lower(col('text')).contains('killed'))

In [12]:
tweets_df_clean = tweets_df_en.select('created_at','id','text','favorite_count','retweet_count','reply_count','retweeted_status_retweet_count','retweeted_status_user_screen_name'
                                      ,tweets_df_en.coordinates_coordinates[0],tweets_df_en.coordinates_coordinates[1],'user_created_at','user_id','user_description'
                                      ,'user_favourites_count','user_followers_count','user_friends_count','user_location'
                                      ,'user_screen_name','user_verified','place_country','place_name','place_full_name'
                                      )
tweets_df_clean = tweets_df_clean.withColumnRenamed("coordinates_coordinates[0]","tweet_longitude")\
                                 .withColumnRenamed("coordinates_coordinates[1]","tweet_latitude")

In [13]:
#Drop tweets without users_description because cannot profile these users
tweets_df_clean = tweets_df_clean.filter(~tweets_df_clean.user_description.isNull())

In [14]:
tweets_df_clean.printSchema()

root
 |-- created_at: string (nullable = true)
 |-- id: long (nullable = true)
 |-- text: string (nullable = true)
 |-- favorite_count: long (nullable = true)
 |-- retweet_count: long (nullable = true)
 |-- reply_count: long (nullable = true)
 |-- retweeted_status_retweet_count: long (nullable = true)
 |-- retweeted_status_user_screen_name: string (nullable = true)
 |-- tweet_longitude: double (nullable = true)
 |-- tweet_latitude: double (nullable = true)
 |-- user_created_at: string (nullable = true)
 |-- user_id: long (nullable = true)
 |-- user_description: string (nullable = true)
 |-- user_favourites_count: long (nullable = true)
 |-- user_followers_count: long (nullable = true)
 |-- user_friends_count: long (nullable = true)
 |-- user_location: string (nullable = true)
 |-- user_screen_name: string (nullable = true)
 |-- user_verified: boolean (nullable = true)
 |-- place_country: string (nullable = true)
 |-- place_name: string (nullable = true)
 |-- place_full_name: string (nu

In [15]:
tweets_df_clean

created_at,id,text,favorite_count,retweet_count,reply_count,retweeted_status_retweet_count,retweeted_status_user_screen_name,tweet_longitude,tweet_latitude,user_created_at,user_id,user_description,user_favourites_count,user_followers_count,user_friends_count,user_location,user_screen_name,user_verified,place_country,place_name,place_full_name
Tue May 24 22:09:...,1529223198478311425,RT @jaketapper: F...,0,0,0,2733,jaketapper,null,null,Fri Jun 24 02:59:...,323002943,Mother/daughter/s...,13661,27,192,Texas,2Short2Sweet,false,null,null,null
Tue May 24 22:09:...,1529223199036059648,RT @meganbang3: M...,0,0,0,23904,meganbang3,null,null,Mon Mar 19 02:44:...,529043291,SHSU ALUMNA | ΣΓΡ...,2158,327,646,null,Lizzy9839,false,null,null,null
Tue May 24 22:09:...,1529223207122653184,RT @ABC: BREAKING...,0,0,0,9094,ABC,null,null,Mon Aug 23 22:17:...,1429930746102390786,Long live the res...,12774,2596,5008,null,TheMastermindX,false,null,null,null
Tue May 24 22:09:...,1529223207164649473,RT @mattxiv: flor...,0,0,0,10238,mattxiv,null,null,Thu Feb 19 04:54:...,21277152,Owner @ajarproduc...,146347,975,1820,San Francisco Bay...,justinputney,false,null,null,null
Tue May 24 22:09:...,1529223207269437440,RT @lopez_wd: 14 ...,0,0,0,2106,lopez_wd,null,null,Tue Oct 15 18:35:...,1184175973815611393,¡Me Vale Winnies!...,13432,159,174,null,valewinnies,false,null,null,null
Tue May 24 22:09:...,1529223210675212288,since it's about ...,0,0,0,null,null,null,null,Tue Nov 27 22:48:...,1067550814904844288,achoo,19293,58,90,kleenex,peepeeegirl,false,null,null,null
Tue May 24 22:09:...,1529223210780221440,RT @mmpadellan: R...,0,0,0,406,mmpadellan,null,null,Mon Jun 28 02:46:...,160405177,Just a simple ner...,63856,660,2082,"Illionos, tinley ...",Uzumakijiraya1,false,null,null,null
Tue May 24 22:09:...,1529223210767720449,RT @Taniel: i wou...,0,0,0,28,Taniel,null,null,Wed Nov 11 19:30:...,89261710,Gay anti-racist s...,50774,862,1255,"Denver, CO",alexcobell,false,null,null,null
Tue May 24 22:10:...,1529223215003754496,RT @NPR: BREAKING...,0,0,0,312,NPR,null,null,Tue Aug 28 07:01:...,786398082,Livin' the good life,3286,277,335,"Bologna, Emilia R...",Lucasn_21,false,null,null,null
Tue May 24 22:10:...,1529223215263928320,RT @TankLawrence:...,0,0,0,3,TankLawrence,null,null,Mon May 21 17:05:...,586699790,26 | Proverbs 16:...,72149,334,333,"Dallas, TX",JMNewski,false,null,null,null


### II. EDA

In [ ]:
#Shape of dataset
print((tweets_df_clean.count()),len(tweets_df_clean.columns))

30334243 22


In [ ]:
from pyspark.sql.functions import col,when, count 
tweets_df_clean.select([count(when(col(c).isNull() , c)).alias(c) for c in tweets_df_clean.columns]
   )

created_at,id,text,favorite_count,retweet_count,reply_count,retweeted_status_retweet_count,retweeted_status_user_screen_name,tweet_longitude,tweet_latitude,user_created_at,user_id,user_description,user_favourites_count,user_followers_count,user_friends_count,user_location,user_screen_name,user_verified,place_country,place_name,place_full_name
0,0,0,0,0,0,10562551,10562551,30279539,30279539,0,0,0,0,0,0,9317222,0,0,30024763,30024763,30024763


In [ ]:
tweets_df_clean.describe(['favorite_count','reply_count','retweeted_status_retweet_count','user_favourites_count','user_followers_count','user_friends_count']).show()

22/12/07 17:16:46 WARN org.apache.spark.deploy.yarn.YarnAllocator: Container from a bad node: container_1670429010405_0001_01_000020 on host: hub-msca-bdp-dphub-students-backup-linhcle-sw-dm6d.c.msca-bdp-students.internal. Exit status: 143. Diagnostics: [2022-12-07 17:16:46.228]Container killed on request. Exit code is 143
[2022-12-07 17:16:46.228]Container exited with a non-zero exit code 143. 
[2022-12-07 17:16:46.230]Killed by external signal
.
22/12/07 17:16:46 WARN org.apache.spark.scheduler.cluster.YarnSchedulerBackend$YarnSchedulerEndpoint: Requesting driver to remove executor 19 for reason Container from a bad node: container_1670429010405_0001_01_000020 on host: hub-msca-bdp-dphub-students-backup-linhcle-sw-dm6d.c.msca-bdp-students.internal. Exit status: 143. Diagnostics: [2022-12-07 17:16:46.228]Container killed on request. Exit code is 143
[2022-12-07 17:16:46.228]Container exited with a non-zero exit code 143. 
[2022-12-07 17:16:46.230]Killed by external signal
.
22/12/07 1

+-------+--------------+-----------+------------------------------+---------------------+--------------------+------------------+
|summary|favorite_count|reply_count|retweeted_status_retweet_count|user_favourites_count|user_followers_count|user_friends_count|
+-------+--------------+-----------+------------------------------+---------------------+--------------------+------------------+
|  count|      30334243|   30334243|                      19771692|             30334243|            30334243|          30334243|
|   mean|           0.0|        0.0|            2226.1909974118553|    44489.19168759214|   9311.808096051713|1548.0279688865155|
| stddev|           0.0|        0.0|             7338.803674721706|    91717.88426749445|  279286.60775152093| 5692.044592065846|
|    min|             0|          0|                             0|                   -1|                  -1|                -1|
|    max|             0|          0|                        516928|              3407404| 

### III. Save to GCP Bucket 

Feature Engineering

In [21]:
tweets_df_clean = tweets_df_clean.\
withColumn("user_type",\
           when(lower(tweets_df_clean.user_description).like('%university%') | lower(tweets_df_clean.user_description).like('%college%'), "University").\
           when(lower(tweets_df_clean.user_description).like('%school%'), "School").\
           when(lower(tweets_df_clean.user_description).like('%department%'), "Government Entities").\
           when(lower(tweets_df_clean.user_description).like('%nonprofit%'), "Nonprofit Organization").\
           when(lower(tweets_df_clean.user_description).like('%news%'), "News Outlet").\
           when(lower(tweets_df_clean.user_description).like('%influencer%')| lower(tweets_df_clean.user_description).like('%social media%'), "Social Media Influencer").\
           when(lower(tweets_df_clean.user_description).like('%cdc%')| lower(tweets_df_clean.user_description).like('%health%') | lower(tweets_df_clean.user_description).like('%disease control%'), "Health Organization").\
           otherwise("Other"))

In [22]:
tweets_df_clean = tweets_df_clean.\
withColumn("tweet_type",\
           when(tweets_df_clean.retweeted_status_retweet_count.isNull(),"original").\
           otherwise("retweet"))

In [23]:
tweets_df_clean

created_at,id,text,favorite_count,retweet_count,reply_count,retweeted_status_retweet_count,retweeted_status_user_screen_name,tweet_longitude,tweet_latitude,user_created_at,user_id,user_description,user_favourites_count,user_followers_count,user_friends_count,user_location,user_screen_name,user_verified,place_country,place_name,place_full_name,user_type,tweet_type
Tue May 24 22:09:...,1529223198478311425,RT @jaketapper: F...,0,0,0,2733,jaketapper,null,null,Fri Jun 24 02:59:...,323002943,Mother/daughter/s...,13661,27,192,Texas,2Short2Sweet,false,null,null,null,Other,retweet
Tue May 24 22:09:...,1529223199036059648,RT @meganbang3: M...,0,0,0,23904,meganbang3,null,null,Mon Mar 19 02:44:...,529043291,SHSU ALUMNA | ΣΓΡ...,2158,327,646,null,Lizzy9839,false,null,null,null,Other,retweet
Tue May 24 22:09:...,1529223207122653184,RT @ABC: BREAKING...,0,0,0,9094,ABC,null,null,Mon Aug 23 22:17:...,1429930746102390786,Long live the res...,12774,2596,5008,null,TheMastermindX,false,null,null,null,Other,retweet
Tue May 24 22:09:...,1529223207164649473,RT @mattxiv: flor...,0,0,0,10238,mattxiv,null,null,Thu Feb 19 04:54:...,21277152,Owner @ajarproduc...,146347,975,1820,San Francisco Bay...,justinputney,false,null,null,null,Other,retweet
Tue May 24 22:09:...,1529223207269437440,RT @lopez_wd: 14 ...,0,0,0,2106,lopez_wd,null,null,Tue Oct 15 18:35:...,1184175973815611393,¡Me Vale Winnies!...,13432,159,174,null,valewinnies,false,null,null,null,Other,retweet
Tue May 24 22:09:...,1529223210675212288,since it's about ...,0,0,0,null,null,null,null,Tue Nov 27 22:48:...,1067550814904844288,achoo,19293,58,90,kleenex,peepeeegirl,false,null,null,null,Other,original
Tue May 24 22:09:...,1529223210780221440,RT @mmpadellan: R...,0,0,0,406,mmpadellan,null,null,Mon Jun 28 02:46:...,160405177,Just a simple ner...,63856,660,2082,"Illionos, tinley ...",Uzumakijiraya1,false,null,null,null,Other,retweet
Tue May 24 22:09:...,1529223210767720449,RT @Taniel: i wou...,0,0,0,28,Taniel,null,null,Wed Nov 11 19:30:...,89261710,Gay anti-racist s...,50774,862,1255,"Denver, CO",alexcobell,false,null,null,null,Other,retweet
Tue May 24 22:10:...,1529223215003754496,RT @NPR: BREAKING...,0,0,0,312,NPR,null,null,Tue Aug 28 07:01:...,786398082,Livin' the good life,3286,277,335,"Bologna, Emilia R...",Lucasn_21,false,null,null,null,Other,retweet
Tue May 24 22:10:...,1529223215263928320,RT @TankLawrence:...,0,0,0,3,TankLawrence,null,null,Mon May 21 17:05:...,586699790,26 | Proverbs 16:...,72149,334,333,"Dallas, TX",JMNewski,false,null,null,null,Other,retweet


In [24]:
tweets_df_verified = tweets_df_clean.filter(tweets_df_clean.user_verified)
tweets_df_coordinates = tweets_df_clean.filter(~tweets_df_clean.tweet_longitude.isNull())

In [63]:
tweets_df_coordinates

created_at,id,text,favorite_count,retweet_count,reply_count,retweeted_status_retweet_count,retweeted_status_user_screen_name,tweet_longitude,tweet_latitude,user_created_at,user_id,user_description,user_favourites_count,user_followers_count,user_friends_count,user_location,user_screen_name,user_verified,place_country,place_name,place_full_name
Tue May 24 22:10:...,1529223332549185539,United Rentals is...,0,0,0,0,null,-96.3119743,30.616559,Thu Mar 05 03:06:...,22870651,Follow this accou...,0,378,232,Texas,tmj_tx_sales,false,United States,College Station,"College Station, TX"
Sun May 29 22:10:...,1531035261727084546,HIGH SCHOOL BOYS ...,0,0,0,0,null,-81.7737592,33.5761134,Sat May 05 03:16:...,571474242,TOURNAMENTS/SHOWC...,927,4779,4112,"Greenville, SC",a1hoopsreport,false,United States,South Carolina,"South Carolina, USA"
Fri May 06 10:35:...,1522525418409971712,Just posted a pho...,0,0,0,0,null,-76.46500996,42.43516201,Wed Oct 21 16:35:...,84111551,The Official Twit...,4561,2711,297,"Poughkeepsie, NY",VassarAthletics,false,United States,East Ithaca,"East Ithaca, NY"
Wed May 25 00:44:...,1529261996146925568,Memory Day 2022 w...,0,0,0,0,null,-86.0027,34.6727,Sat Jul 07 01:25:...,628962945,M.Ed. in Exercise...,4039,417,457,"Scottsboro, AL",BenJohnson_WDE,false,United States,Scottsboro,"Scottsboro, AL"
Thu Jun 02 21:02:...,1532467788060712964,I'm at Collingswo...,0,0,0,0,null,-75.0774181,39.91235286,Mon Aug 26 00:41:...,1700540294,Life is short! Li...,7572,252,446,"Collingswood, NJ",shanemichael70,false,United States,Collingswood,"Collingswood, NJ"
Sun Jun 05 08:42:...,1533368651138494466,Just posted a pho...,0,0,0,0,null,79.453125,20.63278425,Wed Nov 17 05:41:...,1460845489750872064,Official handle o...,20,35,12,Zakir Husain Delh...,NSS_zhdce,false,India,Maharashtra,"Maharashtra, India"
Wed May 25 20:05:...,1529554355808092163,Looking to score ...,0,0,0,0,null,-71.4128343,41.8239891,Wed Aug 18 18:26:...,180047341,Follow this accou...,0,237,197,Rhode Island,tmj_RI_EDU,false,United States,Providence,"Providence, RI"
Wed Aug 17 01:29:...,1559714044914851841,Just posted a pho...,0,0,0,0,null,-96.94709,32.70985,Thu Jun 03 21:13:...,151621374,Over-stressed fat...,8607,8656,2243,"Dallas, TX",_jeremiahj,true,United States,Dallas,"Dallas, TX"
Wed Aug 17 01:30:...,1559714251043897345,Two American foot...,0,0,0,0,null,-99.68322055,31.168839,Fri Oct 24 02:23:...,16941578,An automated samp...,3331,5041,5548,"Houston, TX, USA",NationalStat,false,United States,Texas,"Texas, USA"
Wed May 25 01:51:...,1529278865972924417,College of Nursin...,0,0,0,0,null,79.52951377,29.37578416,Fri Aug 07 16:12:...,1291768967267155969,Official Graphic ...,3,94,7,"Bhimtal, India",gehu_bhimtal,false,India,Bhowali,"Bhowali, India"


In [25]:
bucket_save = 'msca-bdp-students-bucket'

In [ ]:
tweets_df_clean.write.format("parquet").\
mode('overwrite').\
save('gs://' + bucket_save + '/shared_data/linhcle/final_project/clean2')

In [ ]:
tweets_df_verified.write.format("parquet").\
mode('overwrite').\
save('gs://' + bucket_save + '/shared_data/linhcle/final_project/verified')

In [ ]:
tweets_df_coordinates.write.format("parquet").\
mode('overwrite').\
save('gs://' + bucket_save + '/shared_data/linhcle/final_project/coordinates')